In [ ]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [ ]:
import pandas as pd
import numpy as np
import math
import random
from IPython.display import display
from itertools import combinations
from copy import deepcopy
from pprint import pprint as pp

#import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter


pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 11)
pd.set_option('display.width', 230)

In [ ]:
def extract_first_paragraph(text):
    """ 
    Extract the first paragraph from the text as a string.
    It is also removed from the text.
    
    This will need to be rewritten as a class later so that it
    does not have to return the text.
    """
    
    first_paragraph = str()
        
    # iterate through every line of the text
    for position, line in enumerate(text):
                
        # if the line contains only space we stop 
        if not line:                
            text = text[position::]
            break

        # otherwise we pop out the line and add it to the first paragraph
        else:
            first_paragraph=first_paragraph+line
            
    return first_paragraph, text


def extract_heading_definitions(text):
    important_words = {}
        
    # iterate through every line of the text
    for position, line in enumerate(text):
                
        # if the line contains only space we stop 
        if not text[position-2] and not text[position-1]:
            # drop the space at the start and the end of the heading
            heading = line[1:-1]
            important_words[heading] = str()

        # otherwise we pop out the line and add it to the first paragraph
        elif line:
            important_words[heading] += line
        
    return important_words, text


def import_text_and_split_on_spaces(filepath):
    file_object = open(filepath, mode='r')
    # Import the text as a string
    text = file_object.read()
    # Split it into a list where each element is a line in string format
    text = text.splitlines()
    
    return text
    
    
    
def create_lexicon(filepath):
    
    stop_words = set(stopwords.words("english"))
    lemmitizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    
    lexicon=[]
    
    all_words = word_tokenize(important_words['Animals'])
    
    for i in all_words:
        i = i.lower()
        if i not in stop_words:
            if i.isalnum():
                lexicon.append(i)
    print('lexicon:')
    print(lexicon)
    
    lexicon = [lemmitizer.lemmatize(i) for i in lexicon]
    print('\n')
    print('lexicon = [lemmitizer.lemmatize(i) for i in lexicon]:')
    print(lexicon)
    
    
    
    
def extract_important_words(text):
    """
    Given a text in the format of those supplied with the i2x brainteaser, this method: 
    1. extracts the first paragraph as a general definition of the text's subject, (henceforth referred to as the 'class)
    2. takes each following paragraph-title as a feature of the class  
    3. takes each paragraph as a definition of its heading
    """
   
    

    first_paragraph, text = extract_first_paragraph(text)
    important_words, text = extract_heading_definitions(text)
    # ()
    # []
    
    # Get rid of the references, etc.
    for key in important_words.keys():
        if key in ['References', 'Notes', 'External links', 'Further reading', 'See also']:
            important_words.pop(key, None)

    return first_paragraph, important_words

def main():
    filepath = 'script1.txt'
    
    text = import_text_and_split_on_spaces(filepath)
    
    first_paragraph, important_words = extract_important_words(text)
    #print()
    
    print(type(first_paragraph))
    print()
    
    print(important_words['Sweet'])
    print(type(important_words['Sweet']))
    
if __name__=="__main__":
    main()